# 决策树的应用——消费信贷逾期预测
银行在市场经济中发挥着至关重要的作用。他们决定谁能获得融资，以什么条件获得融资。而消费信贷作为银行最重要的贷款之一，主要面对消费者，通过信用卡和小额消费贷款等方式授权银行个人客户短期获得消费资金。

因为各种恶意和非恶意的原因，会发生贷款逾期未还现象。对于银行来说，运用技术手段，如何进行预测和预警就十分重要了。

信用评分算法，利用已有的客户历史数据，对违约的概率进行猜测，是银行用来决定是否应该发放贷款的方法。

<center><img src="image/pexels-photo-7007174.jpeg" alt="image/pexels-photo-7007174.jpeg" width="600"></center>
图片来自Kindel Media


本数据集基于某银行的客户信用情况数据，目标是建立模型预测两年内是否会有90天以上的债务逾期，包含250,000个样本，每个样本包含11个属性，如下表所示


属性  | 定义 
------|------
SeriousDlqin2yrs | 两年内是否有90天以上债务逾期，“1”为有，“0”为无  
RevolvingUtilizationOfUnsecuredLines  | 债务余额占总信用额度比例，除房贷、车贷等分期贷款      
age  | 年龄      
NumberOfTime30-59DaysPastDueNotWorse  | 两年内有过30到59天债务逾期的次数      
DebtRatio  | 月花销（包括债务偿还和生活费）占月收入比例      
MonthlyIncome  | 月收入  
NumberOfOpenCreditLinesAndLoans  | 贷款和信用卡的总数  
NumberOfTimes90DaysLate  | 90天以上债务逾期次数  
NumberRealEstateLoansOrLines  | 抵押贷款和房贷的总数  
NumberOfTime60-89DaysPastDueNotWorse  | 两年内有过60到89天债务逾期的次数  
NumberOfDependents  | 家庭中的受赡养人数（如配偶和孩子）  


该数据集的路径存储在对象`datasets/cs-training.csv`中，而数据集 `datasets/cs-test.csv`作为测试集，用来测试模型的表现。

完成不限于：
1. 描述信贷逾期问题的背景
1. 描述所使用数据集的变量
1. 读取数据集，查看数据集的大小
1. 对数据集进行描述性统计
1. 绘制两年内是否有过90天以上的债务逾期的分布柱状图
1. 以10岁为一个年龄段，绘制每个年龄段的收入分布情况的箱形图
1. 讨论年龄与月花销占月收入比例对债务逾期的影响
1. 绘制两年内是否有过90天以上的债务逾期用户的月收入分布图
1. 对数据进行预处理(例如缺失问题)
1. 将训练集划分为样本内(InSample)和样本外(OutOfSample)
1. 建立决策树模型，对样本内(InSample)的样本进行训练
1. 对样本外(OutOfSample)的数据集进行评估，并优化模型参数
1. 使用测试集测试模型的表现
1. 总结优点和不足

### 1.读取数据集文件
读取客户信用情况数据数据集，查看数据集的前几行。

In [2]:
import pandas as pd
df = pd.read_csv('datasets/cs-training.csv',index_col=0)
df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149996,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149997,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149998,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149999,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


为了方便之后绘图和理解变量，我们将列名从英文改成中文，简化下列名。

In [3]:
EN_CN_dict = {
            'SeriousDlqin2yrs': '是否逾期',              
            'RevolvingUtilizationOfUnsecuredLines': '债务余额占总信用额度比例',              
            'age': '年龄',               
            'NumberOfTime30-59DaysPastDueNotWorse':'30到59天债务逾期的次数',
            'DebtRatio': '月花销占月收入比例',
            'MonthlyIncome': '月收入',               
            'NumberOfOpenCreditLinesAndLoans': '贷款和信用卡的总数',               
            'NumberOfTimes90DaysLate': '90天以上债务逾期次数',               
            'NumberRealEstateLoansOrLines': '抵押贷款和房贷的总数',               
            'NumberOfTime60-89DaysPastDueNotWorse':'60到89天债务逾期的次数',
            'NumberOfDependents': '家庭人数',              
            }

In [4]:
df.columns = [EN_CN_dict[i] for i in df.columns]

In [5]:
df.head(5)

,是否逾期,债务余额占总信用额度比例,年龄,30到59天债务逾期的次数,月花销占月收入比例,月收入,贷款和信用卡的总数,90天以上债务逾期次数,抵押贷款和房贷的总数,60到89天债务逾期的次数,家庭人数
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


### 实验4.1 对数据集进行描述性统计


### 实验4.2 绘制两年内是否有过90天以上的债务逾期的分布柱状图


### 实验4.3 以10岁为一个年龄段，讨论每个年龄段的是否违约情况


### 实验4.4 讨论月花销占月收入比例对债务逾期的影响

### 实验4.5 绘制两年内是否有过90天以上的债务逾期用户的月收入分布图

### 实验4.6 对数据进行预处理(例如缺失问题)


我们直接使用平均值对于缺失的变量进行填充。

### 实验4.7 将训练集划分为样本内(InSample)和样本外(OutOfSample)


### 实验4.8 建立决策树模型，对样本内(InSample)的样本进行训练


### 实验4.9 对样本外(OutOfSample)的数据集进行评估，并优化模型参数


### 实验4.10 使用测试集测试模型的表现，并总结本实验优点和不足
